In [1115]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import re
from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender
import warnings
warnings.filterwarnings('ignore')

In [1116]:
data = pd.read_csv('retail_train.csv')
data.head(2)

In [1117]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### **Задание 1. Weighted Random Recommendation**

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [1118]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity['weight'] = popularity['sales_value'] / popularity['sales_value'].sum()
popularity.head(2)

In [1119]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False)
    
    return recs.tolist()

In [1120]:
%%time

# your_code

items_weights = popularity[['item_id', 'weight']]

result_weighted = pd.DataFrame()
result_weighted['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result_weighted.head()

### **Задание 2. Расчет метрик**
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [1121]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [1122]:
# функция для преобразования строк в список
def str_to_array(result):
    shape_ = result.shape[0]
    for col in result.columns[1:]:    
        for string in range(shape_):
            result[col].iloc[string] = [int(_) for _ in re.findall(r'-?\d+\.?\d*', result[col].iloc[string])]

    return result

In [1123]:
result = pd.read_csv('predictions_basic_new.csv')
str_to_array(result).head(2)

In [1124]:
result['weighted_random_recommendation'] = result_weighted

In [1125]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

### **Задание 3*. Улучшение бейзлайнов и ItemItem**

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [1126]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [1127]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [1128]:
# отберем топ-5000
quant_sum = data_train.groupby('item_id')['quantity'].sum().reset_index()
quant_sum.sort_values(by='quantity', ascending=False, inplace=True)
top_5000 = quant_sum.head(5000)

In [1129]:
%%time

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000['item_id'], n=5))
result.head(2)

In [1130]:
%%time

data_train_top5000 = data_train.loc[data_train['item_id'].isin(top_5000['item_id']), :]
result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train_top5000, n=5))
result.head(2)

In [1131]:
data_train_top5000.head(2)

In [1132]:
popularity_5000 = data_train_top5000.groupby('item_id')['sales_value'].sum().reset_index()
popularity_5000['weight'] = popularity_5000['sales_value'] / popularity_5000['sales_value'].sum()
popularity_5000.head(2)

In [1133]:
%%time

items_weights = popularity_5000[['item_id', 'weight']]
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

In [1134]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

In [1141]:
compare_table_5000 = pd.DataFrame({
    'precision_at_k': [0.0006, 0.006, 0.1552, 0.1552, 0.0212, 0.0374],
    'name_col': ['random_recommendation', 'random_recommendation_5000', 'popular_recommendation', 'popular_recommendation_5000', \
                 'weighted_random_recommendation', 'weighted_random_recommendation_5000']   
})

compare_table_5000

In [1136]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000['item_id']), 'item_id'] = 999999
data_train.head(100)

In [1137]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [1138]:
%%time

k_num = [3, 5, 7]
result_recs = []

for i in k_num:
    
    model = ItemItemRecommender(K=i, num_threads=2) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)
    recs = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    
    result_recs.append(recs)
    
    
result['itemitem_3'] = result_recs[0]
result['itemitem_5'] = result_recs[1]
result['itemitem_7'] = result_recs[2]

In [1139]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

In [1142]:
compare_table_item_k = pd.DataFrame({
    'precision_at_k': [0.1369, 0.1861, 0.1369, 0.1449],
    'name_col': ['itemitem', 'itemitem_3', 'itemitem_5', 'itemitem_7']   
})

compare_table_item_k